# ACS AWS Training: Day 5
## Nathan Miles
06/15/2020

## Outline
- Review of last time
- Docker 101
    - Containers
- Example
    - Create a lambda funciton that will,
        - run SExtractor on an image to identify all sources,
        - generate a catalog of photometry for all sources found in the image
    - Containerize the Lambda function and its dependencies
        - `astropy`
        - `matplotlib`
        - `numpy`
        - `scipy`
        - `sep` (SExtractor in python)
    - Uploading the container to S3
    - Building a Lambda function using the container
    - Invoking the lambda function with `boto3`


## Docker
- Platform for containerizing applications
- Containers are standardized units of software
    - Packages up code and all its dependencies so it runs quickly and reliably in a variety computing environments
        - code, runtime, system tools, system libraries and settings
    - OS-level virtualization

<center><figure><img src="visual_representation_of_a_container.png" width="50%" height="50%" /><figcaption>Credit: <a href="https://www.docker.com/resources/what-container">What is a Container?</a></figcaption></figure></center>

## Example: Create the Lambda Function
- Click to download the following files
    - <a href="./day5_code/build.sh">build.sh</a>
